In [11]:
#Connected Constraints, Renewables, LP without Binary Variables, with A/S, Separate Objective Functions
import pandas as pd
import numpy as np
import math
import sys
from collections import Counter
import timeit
start = timeit.default_timer()
url = None
key = None
from docplex.mp.model import Model
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data_Renewables_Zones.csv')
batt = multi_batts_data.values
solar_profiles = pd.read_csv('solar_profiles.csv')
profiles = solar_profiles.values

#Set Parameters
num_batt = 76  #USER-DEFINED PARAMETER#
batt_name = list(multi_batts_data.columns[10:num_batt+10])
num_hr = len(batt); num_day = math.ceil(num_hr/24)
batt_size = batt[0:num_batt,1].tolist(); batt_cap = batt[0:num_batt,2].tolist(); start_inv = batt[0:num_batt,3].tolist() 
min_re = 0.01*batt[0:num_batt,4]; counties = batt[0:num_batt,5].tolist(); zones = batt[0:num_batt,6].tolist()
rd_pr = batt[:,7].tolist(); ru_pr = batt[:,8].tolist(); sp_pr = batt[:,9].tolist();  re_size = profiles[0:num_batt,1].tolist()
gen_h = {}; re_h = {}; regdn_h = {}; regup_h = {}; spin_h = {}; soc_h = {}; unique_zones = []; header = [""]
for k in range(0,num_batt):
    gen_h[k] = "Gen{0}".format(k+1)
    re_h[k] = "ReCharge{0}".format(k+1)
    regdn_h[k] = "RegDown{0}".format(k+1)
    regup_h[k] = "RegUp{0}".format(k+1)
    spin_h[k] = "Spin{0}".format(k+1)
    soc_h[k] = "SOC{0}".format(k+1)
    header +=[batt_name[k],"","","","",""]
for z in zones:
    if z not in unique_zones:
        unique_zones.append(z)
unique_zones_totals = {}
unique_zones_counter = {}
for z in unique_zones:
    tot_batt_size = 0
    for k in range(0,num_batt):
        if zones[k] == z:
            tot_batt_size += batt_size[k]
    unique_zones_totals[z] = tot_batt_size
    unique_zones_counter[z] = Counter(zones)[z]
eff_c=0.85; eff_re=0.85; krd=0.1; kru=0.1; ksp=0.1; ch_factor = 0.3 #ch_factor is a USER-DEFINED PARAMETER#
gen_val=[0]*num_batt*num_hr; renew_val=[0]*num_batt*num_hr; rd_val=[0]*num_batt*num_hr; ru_val=[0]*num_batt*num_hr 
sp_val=[0]*num_batt*num_hr; soc_val=[0]*num_batt*num_hr; renew_income=[0]*num_batt

#Perform LP optimization
for z in unique_zones:
    m = Model(name='Batt_Sched')
    charge = m.continuous_var_list(num_hr*unique_zones_counter[z])
    disch = m.continuous_var_list(num_hr*unique_zones_counter[z])
    renew = m.continuous_var_list(num_hr*unique_zones_counter[z])
    regdn = m.continuous_var_list(num_hr*unique_zones_counter[z])
    regup = m.continuous_var_list(num_hr*unique_zones_counter[z])
    spin = m.continuous_var_list(num_hr*unique_zones_counter[z])
    soc = m.continuous_var_list(num_hr*unique_zones_counter[z])
    a=0; lmps = []; zone_index = []  #zone_index used to list indices of batteries in zone z
    for k in range(0,num_batt):
        if(zones[k] == z):
            renewable_prof = solar_profiles["{0}".format(counties[k])].values
            renewable_prof_mw = [i*re_size[k]*0.01 for i in renewable_prof]
            lmps += batt[:,10+k].tolist()
            m.add_constraints(renew[a*num_hr+i] <= renewable_prof_mw[i] for i in range(num_hr))
            m.add_constraints(charge[a*num_hr+i]+renew[a*num_hr+i]+disch[a*num_hr+i]+regdn[a*num_hr+i]+regup[a*num_hr+i]+spin[a*num_hr+i] <= batt_size[k] for i in range(num_hr))
            m.add_constraint(eff_c*charge[a*num_hr]+eff_re*renew[a*num_hr]-disch[a*num_hr]+eff_c*krd*regdn[a*num_hr]-kru*regup[a*num_hr]-ksp*spin[a*num_hr]-soc[a*num_hr] == -start_inv[k])
            m.add_constraints(eff_c*charge[a*num_hr+i]+eff_re*renew[a*num_hr+i]-disch[a*num_hr+i]+eff_c*krd*regdn[a*num_hr+i]-kru*regup[a*num_hr+i]-ksp*spin[a*num_hr+i]+soc[a*num_hr+i-1]-soc[a*num_hr+i] == 0 for i in range(1,num_hr))
            m.add_constraints(soc[a*num_hr+i-1] + eff_c*(regdn[a*num_hr+i]+charge[a*num_hr+i]) + eff_re*renew[a*num_hr+i] <= batt_cap[k] for i in range(1,num_hr))
            m.add_constraints(soc[a*num_hr+i-1] - regup[a*num_hr+i] - spin[a*num_hr+i] - disch[a*num_hr+i] >= 0 for i in range(1,num_hr))
            m.add_constraints(soc[a*num_hr+i]<=batt_cap[k] for i in range(num_hr))
            m.add_constraints(m.sum(regdn[a*num_hr+24*j+i]+regup[a*num_hr+24*j+i]+spin[a*num_hr+24*j+i]  for i in range(24)) <= 2.4*batt_size[k] for j in range(num_day))
            m.add_constraints(m.sum(eff_c*charge[a*num_hr+24*j+i]+eff_re*renew[a*num_hr+24*j+i]+eff_c*krd*regdn[a*num_hr+24*j+i]  for i in range(24)) <= batt_cap[k] for j in range(num_day))
            m.add_constraints(m.sum(min_re[k]*(charge[a*num_hr+24*j+i]+krd*regdn[a*num_hr+24*j+i])-(1-min_re[k])*renew[a*num_hr+24*j+i] for i in range(24))<= 0 for j in range(num_day))    #Min Charge from Renewables
            a += 1; zone_index += [k]
    m.add_constraints(m.sum(charge[b*num_hr+i]+renew[b*num_hr+i]+regdn[b*num_hr+i] for b in range(unique_zones_counter[z])) <= unique_zones_totals[z]*ch_factor for i in range(num_hr))
    m.maximize(m.sum(-lmps[i]*(charge[i]+renew[i])+lmps[i]*disch[i]+rd_pr[i%num_hr]*regdn[i]+ru_pr[i%num_hr]*regup[i]+sp_pr[i%num_hr]*spin[i] for i in range(num_hr*unique_zones_counter[z])))
    m.solve()
    for k in range(len(zone_index)):
        gen_val[zone_index[k]*num_hr:(zone_index[k]+1)*num_hr] = [disch[k*num_hr+i].solution_value - charge[k*num_hr+i].solution_value for i in range(num_hr)]
        renew_val[zone_index[k]*num_hr:(zone_index[k]+1)*num_hr] = [renew[k*num_hr+i].solution_value for i in range(num_hr)]
        rd_val[zone_index[k]*num_hr:(zone_index[k]+1)*num_hr] = [regdn[k*num_hr+i].solution_value for i in range(num_hr)]
        ru_val[zone_index[k]*num_hr:(zone_index[k]+1)*num_hr] = [regup[k*num_hr+i].solution_value for i in range(num_hr)]
        sp_val[zone_index[k]*num_hr:(zone_index[k]+1)*num_hr] = [spin[k*num_hr+i].solution_value for i in range(num_hr)]
        soc_val[zone_index[k]*num_hr:(zone_index[k]+1)*num_hr] = [soc[k*num_hr+i].solution_value for i in range(num_hr)]
    del m

#Save Results
sol_values = pd.DataFrame(data={"Date/Time": multi_batts_data["Date/Time"]})
for k in range(0,num_batt):
    sol_values[gen_h[k]] = gen_val[k*num_hr:(k+1)*num_hr]
    sol_values[re_h[k]] = renew_val[k*num_hr:(k+1)*num_hr]
    sol_values[regdn_h[k]] = rd_val[k*num_hr:(k+1)*num_hr]
    sol_values[regup_h[k]] = ru_val[k*num_hr:(k+1)*num_hr]
    sol_values[spin_h[k]] = sp_val[k*num_hr:(k+1)*num_hr]
    sol_values[soc_h[k]] = soc_val[k*num_hr:(k+1)*num_hr]
sol_values.columns = pd.MultiIndex.from_tuples(zip(header,sol_values.columns))
sol_values.to_csv("./multi_batt_result.csv", sep=',',index=False)

stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  394.9960173999989


In [3]:
#A/S to csv
import pandas as pd
import numpy as np
import math
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data_Renewables_Zones.csv')
AS_values = pd.DataFrame(data={"Date/Time": multi_batts_data["Date/Time"]})
AS_header = [""]
num_batt = 56
for k in range(0,num_batt):
    AS_values[regdn_h[k]] = rd_val[k*num_hr:(k+1)*num_hr]
    AS_values[regup_h[k]] = ru_val[k*num_hr:(k+1)*num_hr]
    AS_values[spin_h[k]] = sp_val[k*num_hr:(k+1)*num_hr]
    AS_header +=[batt_name[2*k],"",""]
AS_values.columns = pd.MultiIndex.from_tuples(zip(AS_header,AS_values.columns))
AS_values.to_csv("./AS_Result.csv", sep=',',index=False)

In [6]:
#Profit Calculation, Run After Running Code Above
import pandas as pd
import numpy as np
import math
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data_Renewables_Zones.csv')
batt = multi_batts_data.values
opt_result = pd.read_csv('multi_batt_result.csv')
result = opt_result.values

num_batt = 76; num_hr = len(batt)
batt_name = list(multi_batts_data.columns[10:(num_batt+10)]); zones = batt[0:num_batt,6].tolist()
#renew_size = batt[0:num_batt,4].tolist();
rd_pr = batt[:,7].tolist(); ru_pr = batt[:,8].tolist(); sp_pr = batt[:,9].tolist()
batt_name_list = []; renew_income = []
energy_prof=[0]*num_batt; regdn_income=[0]*num_batt; regup_income=[0]*num_batt; spin_income=[0]*num_batt
for k in range(0,num_batt):
    batt_name_list += [batt_name[k]]
    lmps = batt[:,10+k].tolist()
    gen = result[1:num_hr+1,1+6*k].tolist(); re_charge = result[1:num_hr+1,(2+6*k)].tolist()
    regdn = result[1:(num_hr+1),(3+6*k)].tolist(); regup = result[1:(num_hr+1),(4+6*k)].tolist(); spin = result[1:(num_hr+1),(5+6*k)].tolist()
    gen = list(map(float,gen)); re_charge = list(map(float,re_charge)); regdn = list(map(float,regdn)); regup = list(map(float,regup)); spin = list(map(float,spin))
    energy_prof[k] += sum(x*y for x,y in zip(gen,lmps)) - sum(x*y for x,y in zip(re_charge,lmps))
    regdn_income[k] += sum(x*y for x,y in zip(regdn,rd_pr))
    regup_income[k] += sum(x*y for x,y in zip(regup,ru_pr)) 
    spin_income[k] += sum(x*y for x,y in zip(spin,sp_pr))
    #renewable_prof = renew_size[k]*batt[:,11+2*k]
    #renew_income += [sum([0.01*lmps[i]*renewable_prof[i] for i in range(num_hr)])]
result_profits = pd.DataFrame(data={"Unit Name": batt_name_list, "Zone": zones, "Energy Revenue": energy_prof, "Reg Down Profit": regdn_income, "Reg Up Profit": regup_income, "Spin Profit": spin_income})
result_profits.to_csv("./multi_batt_result_profits.csv", sep=',',index=False)

In [12]:
#Temporal Data Template
import pandas as pd
import numpy as np
import math
opt_result = pd.read_csv('multi_batt_result.csv')
result = opt_result.values
dates = []; date_list = []; prof_name = []; hour_number = {}; generation =[]
num_batt = 76
for d in range(0,num_day):
    dates += [result[24*d+1,0].split(" ",1)[0]]
for k in range(0,num_batt):
    prof_name += [batt_name[k]]*len(dates)
    date_list += dates
batt_gen = pd.DataFrame(data={"Profile Name": prof_name, "Date": date_list})
for h in range(1,25):
    generation =[]
    hour_number[h-1] = "{0}".format(h)
    for k in range(0,num_batt):
        for d in range(0,num_day):
            generation += [float(result[24*d+h,6*k+1])-float(result[24*d+h,6*k+2])]
    batt_gen["{0}".format(h)]=generation
batt_gen.to_csv("./batt_gen.csv", sep=',',index=False)